In [12]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import ast
import time

try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

dir_base_model = "/content/gdrive/My Drive/DeepLearning/2d-mapper/mapper-2dt"
tf.enable_v2_behavior()
print(tf.__version__)

2.1.0-rc1


In [13]:
# Load input data
from google.colab import drive
drive.mount('/content/gdrive')
# ("X","Y","S","T")) in csv
data = pd.read_csv(dir_base_model + '.csv', index_col=False)

v = int(len(data) * 0.8)
train_data = data[1:v]
test_data = data[v:len(data)]
x_train = train_data[["S", "T"]]
y_train = train_data[["X", "Y"]]
x_test = test_data[["S", "T"]]
y_test = test_data[["X", "Y"]]
x_train.columns = [''] * len(x_train.columns)
y_train.columns = [''] * len(y_train.columns)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def load_last_and_train():
  model = tf.keras.models.load_model(dir_base_model + '.model')
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
  start_time = time.time()
  model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
  #model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
  model.fit(x = x_train.values, y = y_train.values, verbose=0, epochs=1000)#, batch_size=9)
  result = model.evaluate(x_test.values,  y_test.values, verbose=0)
  end_time = time.time() - start_time
  print(str(result) + " taken " + str(end_time) + " seconds")
  return model

In [0]:
def save_if_better(model):
  try:
      f = open(dir_base_model + '-info.txt', 'rt')
      f.close()
      exists = True
  except FileNotFoundError:
      exists = False
  # Check if saved model is better. If false, save this one.
  if (exists == True):
    file = open(dir_base_model + '-info.txt', 'rt')
    value = file.read()
    file.close()
    old_evaluation_result = ast.literal_eval(value)
    print("Saved loss is: " + str(old_evaluation_result))
  # Evaluate model
  evaluation_result = model.evaluate(x_test.values,  y_test.values, verbose=0)
  # Save only if found a better configuration
  if (exists == False or evaluation_result[0] < old_evaluation_result[0]):
    print("Found better model with loss: " + str(evaluation_result))
    file = open(dir_base_model + '-info.txt', 'wt')
    file.write(str(evaluation_result))
    file.close()
    model.save(dir_base_model + '.model')

In [21]:
for x in range(10):
  model = load_last_and_train()
  save_if_better(model)

 # Get final weights
print("Current weights:\n")
print(model.get_weights())

[10.648910522460938, 2.8353703, 10.6489105] taken 3.251591682434082 seconds
Saved loss is: [10.60622501373291, 2.8289435, 10.606225]
[10.648869514465332, 2.8353631, 10.6488695] taken 3.0984249114990234 seconds
Saved loss is: [10.60622501373291, 2.8289435, 10.606225]
[10.648900032043457, 2.8353686, 10.6489] taken 3.033210277557373 seconds
Saved loss is: [10.60622501373291, 2.8289435, 10.606225]
[10.648881912231445, 2.835366, 10.648882] taken 3.132598400115967 seconds
Saved loss is: [10.60622501373291, 2.8289435, 10.606225]
[10.648896217346191, 2.835369, 10.648896] taken 3.170327663421631 seconds
Saved loss is: [10.60622501373291, 2.8289435, 10.606225]
[10.648895263671875, 2.835368, 10.648895] taken 3.063469886779785 seconds
Saved loss is: [10.60622501373291, 2.8289435, 10.606225]
[10.648882865905762, 2.8353674, 10.648883] taken 3.039896249771118 seconds
Saved loss is: [10.60622501373291, 2.8289435, 10.606225]
[10.648895263671875, 2.835368, 10.648895] taken 3.0269360542297363 seconds
Sav